# Q1.
해당 경우는 Acceptance rate가 높아서 가입이 쉽다. 

Type 1 error는 FN와 관련이 있고 (가입해줘야 할 사람을 가입 거부), Type 2 error는 FP과 관계가 있다 (가입을 해주면 안될 사람을 가입해주는 것).

Type 1 error = $P(reject\ H_0 | H_0\ is\ true)$ <br>
Type 2 error = $P(accept\ H_0 | H_0 is\ not\ true)$

Acceptance rate이 높기 때문에 type 2 error가 높을 것이라고 생각한다.

# Q2-1. 
Accuracy = $\frac{TP+TN}{TP+FN+FP+TN}$, 모델이 옳게 예측한 비율 <br>
Precision = $\frac{TP}{TP+FP}$, 모델이 True라고 분류한 것 중에 실제 True인 것의 비율 <br>
Recall = $\frac{TP}{TP+FN}$, 실제 True인 것 중에서 모델이 True라고 예측한 것의 비율 <br>

# Q2-2. 
CCTV 영상 데이터를 통해 범죄를 예측할 때는 Recall 값이 낮다면 안된다고 생각합니다. 즉, 실제 범죄가 일어났는데 잘못 예측하여 범죄가 일어나지 않았다고 한다면 큰 문제가 되기 때문입니다. 
이 경우에 threshold를 높게 설정한다면, 실제 범죄를 잘못 예측하는 경우가 생길 것이라고 생각합니다. Precision은 높지만, Recall이 낮아질 수 있다고 생각합니다. 반면 낮게 설정한다면 인력 낭비가 심해질 것 같습니다. 따라서 trade off 중에서 최적해를 찾아야 한다고 생각합니다.

# Q3.

In [1]:
import torch 
import torch.nn as nn 
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
from torch.autograd import Variable 

In [2]:
# model에 사용할 변수들 값 설정
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset (Images and Labels) 
# Training data
train_dataset = dsets.MNIST(root ='./data',  
                            train = True,  
                            transform = transforms.ToTensor(), 
                            download = True) 

# Test data 
test_dataset = dsets.MNIST(root ='./data',  
                           train = False,  
                           transform = transforms.ToTensor()) 
  
# Dataset Loader (Input Pipline) 
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,  
                                           batch_size = batch_size,  
                                           shuffle = True) 
  
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,  
                                          batch_size = batch_size,  
                                          shuffle = False)

In [4]:
# 학습에 사용할 함수 정의
class LogisticRegression(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(LogisticRegression, self).__init__() 
        self.linear = nn.Linear(input_size, num_classes) 
  
    def forward(self, x): 
        out = self.linear(x) 
        return out 

In [5]:
# 모델 설정
model = LogisticRegression(input_size, num_classes) 

In [6]:
criterion = nn.CrossEntropyLoss() # loss fuction으로 cross entropy 설정 
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # optimizer로 SGD 설정

In [7]:
# Training the Model 
for epoch in range(num_epochs): 
    for i, (images, labels) in enumerate(train_loader): 
        images = Variable(images.view(-1, 28 * 28)) # 1차원으로 차원변환
        labels = Variable(labels) 
  
        # Forward + Backward + Optimize 
        optimizer.zero_grad() # gradient 초기화
        outputs = model(images) 
        loss = criterion(outputs, labels) # 상기된 output을 다시 input으로 설정
        loss.backward() # back propagation
        optimizer.step() 
        
        # 진행상황
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train_dataset) // batch_size, loss.data))

Epoch: [ 1/ 5], Step: [ 100/ 600], Loss: 2.1803
Epoch: [ 1/ 5], Step: [ 200/ 600], Loss: 2.1045
Epoch: [ 1/ 5], Step: [ 300/ 600], Loss: 1.9849
Epoch: [ 1/ 5], Step: [ 400/ 600], Loss: 1.9503
Epoch: [ 1/ 5], Step: [ 500/ 600], Loss: 1.8939
Epoch: [ 1/ 5], Step: [ 600/ 600], Loss: 1.7511
Epoch: [ 2/ 5], Step: [ 100/ 600], Loss: 1.6651
Epoch: [ 2/ 5], Step: [ 200/ 600], Loss: 1.6867
Epoch: [ 2/ 5], Step: [ 300/ 600], Loss: 1.6427
Epoch: [ 2/ 5], Step: [ 400/ 600], Loss: 1.5426
Epoch: [ 2/ 5], Step: [ 500/ 600], Loss: 1.5352
Epoch: [ 2/ 5], Step: [ 600/ 600], Loss: 1.4624
Epoch: [ 3/ 5], Step: [ 100/ 600], Loss: 1.3806
Epoch: [ 3/ 5], Step: [ 200/ 600], Loss: 1.4145
Epoch: [ 3/ 5], Step: [ 300/ 600], Loss: 1.3709
Epoch: [ 3/ 5], Step: [ 400/ 600], Loss: 1.2260
Epoch: [ 3/ 5], Step: [ 500/ 600], Loss: 1.3233
Epoch: [ 3/ 5], Step: [ 600/ 600], Loss: 1.2020
Epoch: [ 4/ 5], Step: [ 100/ 600], Loss: 1.2486
Epoch: [ 4/ 5], Step: [ 200/ 600], Loss: 1.1087
Epoch: [ 4/ 5], Step: [ 300/ 600], Loss:

In [8]:
# Test the Model 
correct = 0
total = 0
for images, labels in test_loader: 
    images = Variable(images.view(-1, 28 * 28)) 
    outputs = model(images) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total)) 

Accuracy of the model on the 10000 test images:  82 %


..\aten\src\ATen\native\BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.


# Q4-1.

optimizer 함수가 아닌 스스로 중요도를 초기화하는 구문과 업데이트를 진행하는 구문을 추가해야한다. $w=w-lr*w_0,\ where\ w_0\ is\ inital\ value\ and\ lr\ is\ learning\ rate$

# Q4-2.

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[1], [2], [3]])

class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)
        
    def forward(self, x):
        return self.linear(x)
    
# 모델 초기화
model1 = LinearRegressionModel()
model2 = LinearRegressionModel()
model3 = LinearRegressionModel()

In [10]:
# Adam
optimizer_Adam = optim.Adam(model1.parameters(), lr=0.01)
#RMSprop
optimizer_R = optim.RMSprop(model2.parameters(), lr=0.01)
#Adagrad
optimizer_Adag = optim.Adagrad(model3.parameters(), lr=0.01)

In [11]:
nb_epochs = 1000

for epoch1 in range(nb_epochs + 1):

    # H(x) 계산
    prediction1 = model1(x_train)
    
    # cost 계산
    cost1 = F.mse_loss(prediction1, y_train)
    
    # cost로 H(x) 개선
    optimizer_Adam.zero_grad() 
    cost1.backward()
    optimizer_Adam.step()
    
    if epoch1 % 100 == 0:
        params1 = list(model1.parameters())
        W1 = params1[0].item()
        b1 = params1[1].item()
        
        print('Epoch {:4d}/{} W1: {:.3f}, b1: {:.3f} Cost1: {:.6f}'.format(epoch1, nb_epochs, W1, b1, cost1.item()))


Epoch    0/1000 W1: 0.359, b1: 0.492 Cost1: 0.953734
Epoch  100/1000 W1: 0.713, b1: 0.629 Cost1: 0.058427
Epoch  200/1000 W1: 0.820, b1: 0.397 Cost1: 0.023203
Epoch  300/1000 W1: 0.903, b1: 0.214 Cost1: 0.006757
Epoch  400/1000 W1: 0.955, b1: 0.100 Cost1: 0.001483
Epoch  500/1000 W1: 0.982, b1: 0.041 Cost1: 0.000248
Epoch  600/1000 W1: 0.993, b1: 0.015 Cost1: 0.000032
Epoch  700/1000 W1: 0.998, b1: 0.005 Cost1: 0.000003
Epoch  800/1000 W1: 0.999, b1: 0.001 Cost1: 0.000000
Epoch  900/1000 W1: 1.000, b1: 0.000 Cost1: 0.000000
Epoch 1000/1000 W1: 1.000, b1: 0.000 Cost1: 0.000000


In [12]:
for epoch2 in range(nb_epochs + 1):
    prediction2 = model2(x_train)
    
    cost2 = F.mse_loss(prediction2, y_train)
    
    optimizer_R.zero_grad() 
    cost2.backward()
    optimizer_R.step()
    
    if epoch2 % 100 == 0:
        params2 = list(model2.parameters())
        W2 = params2[0].item()
        b2 = params2[1].item()
        
        print('Epoch {:4d}/{} W2: {:.3f}, b2: {:.3f} Cost2: {:.6f}'.format(epoch2, nb_epochs, W2, b2, cost2.item()))

Epoch    0/1000 W2: -0.522, b2: 0.500 Cost2: 9.844289
Epoch  100/1000 W2: 0.422, b2: 1.244 Cost2: 0.231430
Epoch  200/1000 W2: 0.549, b2: 0.997 Cost2: 0.145514
Epoch  300/1000 W2: 0.689, b2: 0.689 Cost2: 0.069671
Epoch  400/1000 W2: 0.828, b2: 0.382 Cost2: 0.021504
Epoch  500/1000 W2: 0.933, b2: 0.148 Cost2: 0.003241
Epoch  600/1000 W2: 0.986, b2: 0.031 Cost2: 0.000146
Epoch  700/1000 W2: 0.999, b2: 0.002 Cost2: 0.000001
Epoch  800/1000 W2: 1.000, b2: 0.000 Cost2: 0.000000
Epoch  900/1000 W2: 1.012, b2: 0.012 Cost2: 0.002519
Epoch 1000/1000 W2: 1.000, b2: 0.000 Cost2: 0.000001


In [13]:
for epoch3 in range(nb_epochs + 1):    
    prediction3 = model3(x_train)

    cost3 = F.mse_loss(prediction3, y_train)
    
    optimizer_Adag.zero_grad() 
    cost3.backward()
    optimizer_Adag.step()

    if epoch3 % 100 == 0:       
        params3 = list(model3.parameters())
        W3 = params3[0].item()
        b3 = params3[1].item()
        
        print('Epoch {:4d}/{} W3: {:.3f}, b3: {:.3f} Cost3: {:.6f}'.format(epoch3, nb_epochs, W3, b3, cost3.item()))

Epoch    0/1000 W3: 0.983, b3: 0.193 Cost3: 0.035583
Epoch  100/1000 W3: 0.948, b3: 0.117 Cost3: 0.001984
Epoch  200/1000 W3: 0.959, b3: 0.092 Cost3: 0.001235
Epoch  300/1000 W3: 0.967, b3: 0.073 Cost3: 0.000775
Epoch  400/1000 W3: 0.974, b3: 0.058 Cost3: 0.000490
Epoch  500/1000 W3: 0.979, b3: 0.046 Cost3: 0.000310
Epoch  600/1000 W3: 0.984, b3: 0.037 Cost3: 0.000197
Epoch  700/1000 W3: 0.987, b3: 0.029 Cost3: 0.000125
Epoch  800/1000 W3: 0.990, b3: 0.023 Cost3: 0.000080
Epoch  900/1000 W3: 0.992, b3: 0.019 Cost3: 0.000051
Epoch 1000/1000 W3: 0.993, b3: 0.015 Cost3: 0.000032
